# Theorem 4.2: Reading the output from C++

The file `j.txt` in the `generated_antichains` directory is generated by running `./cluster j`, and contains two lines corresponding to the $j$-th coefficient of the cluster expansion. More precisely, the sum of the two lines in `j.txt` equals
$$\sum_{\substack{\Gamma \in \mathcal{C} \\ \lVert \Gamma \rVert = j}} w(\Gamma)$$
as defined in the proof of Theorem 4.2. Instead of considering the three central layers, however, we consider three consecutive layers $k-1$, $k$ and $k+1$ of an $n$-dimensional hypercube, where $k$ is a parameter and layers $k-1$ and $k+1$ together constitute the defect side. 
Therefore, each of the two lines is a function of $n$, $k$ and $\lambda$ (denoted by $W^1_j(n, k, \lambda)$ and $W^2_j(n, k, \lambda)$ respectively), and the following holds:

* Every term in the first line is multiplied by $\binom{n}{k-1}$, and it corresponds to the contribution (to the above displayed sum) of fixing a root in the $(k-1)$-th layer. The weight of each cluster is already divided by $\ell$, the number of vertices in the cluster (without multiplicity).
* Similarly, every term in the second line is multiplied by $\binom{n}{k+1}$ and corresponds to the contribution of fixing a root in the $(k+1)$-th layer.

The functions computed by the C++ program are not polynomials, since it contains exponents which are multiples of $2n-k$. However, when $k = \lceil n/2 \rceil$, then they are polynomials in $n$.

We start by reading the files into the variable `w`:

In [1]:
%display latex
import pathlib

W = {}
var("λ k n")

for f in sorted(pathlib.Path('generated-antichains').glob('*.txt')):
    j = int(f.name[:-4])
    W[j] = [sage_eval(line, locals={'λ': λ, 'k': k, 'n': n}).collect(binom)
            for line, binom in zip(f.read_text().strip().split("\n"),
                                   [binomial(n, k-1), binomial(n, k+1)])]

For example, the contribution coming from clusters $\Gamma$ with $\lVert \Gamma \rVert = 2$ is the sum of the two lines below:

In [2]:
display(W[2][0])
display(W[2][1])

-1/4*(2*(k - n - 1)*(k - 1)*(λ^2/(λ + 1)^(-2*k + 2*n + 1) - λ^2/(λ + 1)^(-2*k + 2*n + 2)) + (k^2 - 2*k*n + n^2 - k + n)*λ^2/((λ + 1)^(k + 1)*(λ + 1)^(-k + n + 1)) + 2*λ^2/(λ + 1)^(-2*k + 2*n + 2))*binomial(n, k - 1)

-1/4*(2*(k - n + 1)*(k + 1)*(λ^2/(λ + 1)^(2*k + 1) - λ^2/(λ + 1)^(2*k + 2)) + (k^2 + k)*λ^2/((λ + 1)^(k + 1)*(λ + 1)^(-k + n + 1)) + 2*λ^2/(λ + 1)^(2*k + 2))*binomial(n, k + 1)

The coefficients given by `w[j](n, k, λ)` are valid for any choice of $2j < k < n-2j$. Let us do a minor sanity check of this statement in the context of "central layers":
* When $n = 2m+1$, there are two definitions for "three central layers": either layers $m-1$, $m$ and $m+1$, or layers $m$, $m+1$ and $m+2$. Fixing a root in layer $m-1$ in the former should give exactly the same contribution from fixing a vertex in layer $m+2$ in the latter.
* When $n = 2m$, the contribution from fixing a root in layer $m-1$ should equal the contribution from fixing a root in layer $m+1$.

In [3]:
var("m")
for j, (kminus, kplus) in W.items():
    if j <= 5: # for performance
        assert bool(kminus.subs(n == 2*m+1).subs(k == m) == 
                    kplus.subs(n == 2*m+1).subs(k == m+1))
        assert bool(kminus.subs(n == 2*m  ).subs(k == m) == 
                    kplus.subs(n == 2*m  ).subs(k == m))

The `display_table` function below has no mathematical content, but it helps visualize the output later on. Ignore the following cell.

In [4]:
import collections.abc
import functools

EVEN, ODD = (k == n/2), (k == (n+1)/2)

def clean(h, ex):
    return ex.subs(h).simplify_rational()

def display_table(limit, fn, fn_name, merge_even=lambda x: x):
    from IPython.display import Markdown
    for hyp, label, merge in [(EVEN, "Even case", merge_even),
                              (ODD, "Odd case", lambda x: x)]:
        display(Markdown(f"***{label}***"))
        results = fn(hyp)
        if "," not in fn_name:
            results = [results]
            
        for out, lbl in zip(results, fn_name.split(",")):
            if isinstance(out, dict):
                for j, val in out.items():
                    if j <= limit:
                        val = merge(val)
                        if isinstance(val, list):
                            for idx, v in enumerate(val):
                                display(f"{lbl}^{idx+1}_{j} = " + latex(v))
                        else:
                            display(f"{lbl}_{j} = " + latex(val))
                        display("")
            else:
                display(f"{lbl} = " + latex(out))

# https://wiki.python.org/moin/PythonDecoratorLibrary#Memoize
class memoized(object):
   def __init__(self, func):
      self.func = func
      self.cache = {}
   def __call__(self, *args):
      if not isinstance(args, collections.abc.Hashable):
         return self.func(*args)
      if args in self.cache:
         return self.cache[args]
      else:
         value = self.func(*args)
         self.cache[args] = value
         return value
   def __repr__(self):
      return self.func.__doc__
   def __get__(self, obj, objtype):
      return functools.partial(self.__call__, obj)

# Theorem 4.2: The polynomials $S_j$ and $P_j$

The main goal of this section is to obtain polynomials $S^{1}_j(n, \lambda)$ and $S^{2}_j(n, \lambda)$. The paper does only the odd case, noting that the "case of $n$ even is analogous (and simpler)". To accommodate for both cases, $S^1$ and $S^2$ will be defined regardless of the parity of $n$, although they are defined by different formulas in the even and odd cases. Moreover, $S^{1}_j = S^{2}_j$ when $n$ is even, and $S^{0}_j$ in this case is just $S^{1}_j + S^{2}_j$.

We will choose $k = \lceil n/2 \rceil$ to compute the $S$ polynomials.

Recall that the $S^r_j$ are polynomials such that $\operatorname{deg}_n S^r_j \leq 2(j-1)$, $\operatorname{deg}_\lambda S^r_j \leq 2j^2$ and
$$\sum_{\substack{\Gamma \in \mathcal{C} \\ \lVert \Gamma \rVert = j}} w_C(\Gamma) = \binom{n}{k-1}S^1_j(n, \lambda)(1+\lambda)^{-j(k+1)} + \binom{n}{k+1}S^2_j(n, \lambda)(1+\lambda)^{-j(k+1)}.$$

Note that $S^r_j$ is then just $W^r_j(n, k, \lambda) \cdot (1+\lambda)^{j(k+1)}$ for $k = \lceil n/2 \rceil$ (that is, $k = n/2$ when $n$ is even or $k = (n+1)/2$ when $n$ is odd) divided by the appropriate binomial ($\binom{n}{k-1}$ or $\binom{n}{k+1}$).

In [5]:
@memoized
def compute_S(h):
    S = {}
    for j in W.keys():
        S[j] = [W[j][0] / binomial(n, k-1), W[j][1] / binomial(n, k+1)]
        S[j] = [clean(h, ex * (1+λ)^(j * (k+1))) for ex in S[j]]
        assert all(QQ['λ']['n'](poly).degree() <= 2*(j-1) for poly in S[j])
        assert all(QQ['n']['λ'](poly).degree() <= 2*j^2 for poly in S[j])
    return S

display_table(6, compute_S, "S", merge_even=sum)

***Even case***

S_1 = 2 \, λ

''

S_2 = \frac{1}{4} \, {\left(n^{2} - 4\right)} λ^{3} - \frac{1}{8} \, {\left(n^{2} + 2 \, n + 8\right)} λ^{2}

''

...

S^2_5 = -\frac{1}{3840} \, {\left(n^{6} - 7 \, n^{5} + 6 \, n^{4} + 70 \, n^{3} - 151 \, n^{2} - 63 \, n + 144\right)} λ^{15} - \frac{1}{384} \, {\left(n^{6} - 7 \, n^{5} + 6 \, n^{4} + 70 \, n^{3} - 151 \, n^{2} - 63 \, n + 144\right)} λ^{14} - \frac{1}{1280} \, {\left(2 \, n^{7} + 6 \, n^{6} - 111 \, n^{5} + 175 \, n^{4} + 928 \, n^{3} - 2296 \, n^{2} - 819 \, n + 2115\right)} λ^{13} - \frac{1}{15360} \, {\left(16 \, n^{8} + 73 \, n^{7} - 377 \, n^{6} - 2847 \, n^{5} + 9991 \, n^{4} + 21347 \, n^{3} - 80163 \, n^{2} - 18573 \, n + 70533\right)} λ^{12} - \frac{1}{15360} \, {\left(76 \, n^{8} - 35 \, n^{7} - 1991 \, n^{6} - 1917 \, n^{5} + 29139 \, n^{4} + 19075 \, n^{3} - 184289 \, n^{2} - 24803 \, n + 149385\right)} λ^{11} - \frac{1}{61440} \, {\left(513 \, n^{8} - 1920 \, n^{7} - 13036 \, n^{6} + 19848 \, n^{5} + 159446 \, n^{4} - 126288 \, n^{3} - 994732 \, n^{2} + 675144 \, n + 1414593\right)} λ^{10} - \frac{1}{491520} \, {\left(1175 \, n^{8} - 9360 \, n^{7} - 77972 \, n^{6} - 7280 \, n^{5} + 1354386 \, n^{4} + 1333136 \, n^{3} - 6813892 \, n^{2} - 4073104 \, n + 2288175\right)} λ^{9} + \frac{1}{122880} \, {\left(189 \, n^{8} + 1128 \, n^{7} + 7356 \, n^{6} - 28376 \, n^{5} - 319394 \, n^{4} - 77832 \, n^{3} + 1859740 \, n^{2} + 1295992 \, n - 111219\right)} λ^{8} + \frac{1}{245760} \, {\left(459 \, n^{8} - 2592 \, n^{7} - 17860 \, n^{6} - 6720 \, n^{5} - 32406 \, n^{4} + 76256 \, n^{3} + 1241164 \, n^{2} + 1573504 \, n + 940611\right)} λ^{7} - \frac{1}{61440} \, {\left(88 \, n^{8} + 278 \, n^{7} - 779 \, n^{6} - 5846 \, n^{5} - 2475 \, n^{4} - 4086 \, n^{3} - 114249 \, n^{2} - 125002 \, n - 78681\right)} λ^{6} + \frac{1}{491520} \, {\left(125 \, n^{8} + 1184 \, n^{7} + 2716 \, n^{6} + 7392 \, n^{5} + 25406 \, n^{4} + 34400 \, n^{3} + 78268 \, n^{2} + 92192 \, n + 126957\right)} λ^{5}

''

S^1_6 = \frac{1}{46080} \, {\left(n^{6} - 16 \, n^{5} + 85 \, n^{4} - 160 \, n^{3} + 19 \, n^{2} + 176 \, n - 105\right)} λ^{27} + \frac{7}{15360} \, {\left(n^{6} - 16 \, n^{5} + 85 \, n^{4} - 160 \, n^{3} + 19 \, n^{2} + 176 \, n - 105\right)} λ^{26} + \frac{7}{1536} \, {\left(n^{6} - 16 \, n^{5} + 85 \, n^{4} - 160 \, n^{3} + 19 \, n^{2} + 176 \, n - 105\right)} λ^{25} + \frac{133}{4608} \, {\left(n^{6} - 16 \, n^{5} + 85 \, n^{4} - 160 \, n^{3} + 19 \, n^{2} + 176 \, n - 105\right)} λ^{24} + \frac{1}{3072} \, {\left(n^{7} + 384 \, n^{6} - 6299 \, n^{5} + 33696 \, n^{4} - 63629 \, n^{3} + 7680 \, n^{2} + 69927 \, n - 41760\right)} λ^{23} + \frac{1}{15360} \, {\left(80 \, n^{7} + 5581 \, n^{6} - 101716 \, n^{5} + 559045 \, n^{4} - 1068520 \, n^{3} + 136879 \, n^{2} + 1170156 \, n - 701505\right)} λ^{22} + \frac{1}{184320} \, {\left(7295 \, n^{7} + 108977 \, n^{6} - 2868117 \, n^{5} + 16917125 \, n^{4} - 33294675 \, n^{3} + 4847843 \, n^{2} + 36155497 \, n - 21873945\right)} λ^{21} + \frac{1}{12288} \, {\left(24 \, n^{8} + 1999 \, n^{7} - 655 \, n^{6} - 318613 \, n^{5} + 2192197 \, n^{4} - 4559043 \, n^{3} + 807747 \, n^{2} + 4875657 \, n - 2999313\right)} λ^{20} + \frac{1}{12288} \, {\left(324 \, n^{8} + 3801 \, n^{7} - 33925 \, n^{6} - 351171 \, n^{5} + 3337359 \, n^{4} - 7585669 \, n^{3} + 1702369 \, n^{2} + 7933039 \, n - 5006127\right)} λ^{19} + \frac{1}{36864} \, {\left(216 \, n^{9} + 3190 \, n^{8} + 951 \, n^{7} - 243239 \, n^{6} - 602461 \, n^{5} + 12265753 \, n^{4} - 31621579 \, n^{3} + 9008235 \, n^{2} + 32222873 \, n - 21033939\right)} λ^{18} + \frac{1}{368640} \, {\left(648 \, n^{10} + 8640 \, n^{9} + 21455 \, n^{8} - 328990 \, n^{7} - 2743034 \, n^{6} + 1190858 \, n^{5} + 125973748 \, n^{4} - 377260858 \, n^{3} + 132027170 \, n^{2} + 376390350 \, n - 255279987\right)} λ^{17} + \frac{1}{368640} \, {\left(3888 \, n^{10} + 2445 \, n^{9} - 67210 \, n^{8} - 458050 \, n^{7} - 1265158 \, n^{6} + 4431080 \, n^{5} + 113677854 \, n^{4} - 389296958 \, n^{3} + 158526734 \, n^{2} + 385321483 \, n - 270876108\right)} λ^{16} + \frac{1}{1474560} \, {\left(35755 \, n^{10} - 151160 \, n^{9} - 509255 \, n^{8} + 170480 \, n^{7} + 686670 \, n^{6} + 5544256 \, n^{5} + 380169042 \, n^{4} - 1388697712 \, n^{3} + 618053431 \, n^{2} + 1383134136 \, n - 998435643\right)} λ^{15} + \frac{1}{1474560} \, {\left(36185 \, n^{10} - 279295 \, n^{9} - 230050 \, n^{8} + 2518868 \, n^{7} + 64438 \, n^{6} - 12841666 \, n^{5} + 314606448 \, n^{4} - 1110545020 \, n^{3} + 504316873 \, n^{2} + 1121147113 \, n - 818793894\right)} λ^{14} + \frac{1}{2949120} \, {\left(20005 \, n^{10} - 278610 \, n^{9} - 69939 \, n^{8} + 4698744 \, n^{7} - 3079126 \, n^{6} - 40392188 \, n^{5} + 516318914 \, n^{4} - 1690039848 \, n^{3} + 750657553 \, n^{2} + 1726011902 \, n - 1263847407\right)} λ^{13} - \frac{1}{1474560} \, {\left(5493 \, n^{10} - 65410 \, n^{9} + 298607 \, n^{8} - 817136 \, n^{7} - 984326 \, n^{6} + 18762460 \, n^{5} - 184533018 \, n^{4} + 588611616 \, n^{3} - 268032447 \, n^{2} - 606491530 \, n + 453491451\right)} λ^{12} + \frac{1}{23592960} \, {\left(29827 \, n^{10} + 841840 \, n^{9} - 6025391 \, n^{8} + 1559232 \, n^{7} + 56016734 \, n^{6} - 233132768 \, n^{5} + 1614355682 \, n^{4} - 5312904256 \, n^{3} + 2716869711 \, n^{2} + 5543635952 \, n - 4404839523\right)} λ^{11} + \frac{1}{23592960} \, {\left(68935 \, n^{10} - 469680 \, n^{9} - 1468707 \, n^{8} + 1838144 \, n^{7} + 39609478 \, n^{6} - 153244832 \, n^{5} + 727187578 \, n^{4} - 2327310528 \, n^{3} + 1312576419 \, n^{2} + 2479186896 \, n - 2136956103\right)} λ^{10} - \frac{1}{5898240} \, {\left(4975 \, n^{10} + 54500 \, n^{9} - 248783 \, n^{8} - 184144 \, n^{7} - 2295706 \, n^{6} + 17921432 \, n^{5} - 87407534 \, n^{4} + 236173104 \, n^{3} - 112872389 \, n^{2} - 253964892 \, n + 222480237\right)} λ^{9} - \frac{1}{1474560} \, {\left(945 \, n^{10} - 9375 \, n^{9} - 14286 \, n^{8} + 46164 \, n^{7} - 96794 \, n^{6} + 1165118 \, n^{5} - 7286216 \, n^{4} + 20670212 \, n^{3} - 7544159 \, n^{2} - 21872119 \, n

S^2_6 = -\frac{1}{184320} \, {\left(5 \, n^{7} - 69 \, n^{6} + 297 \, n^{5} + 55 \, n^{4} - 3425 \, n^{3} + 5729 \, n^{2} + 3123 \, n - 5715\right)} λ^{21} - \frac{1}{12288} \, {\left(5 \, n^{7} - 69 \, n^{6} + 297 \, n^{5} + 55 \, n^{4} - 3425 \, n^{3} + 5729 \, n^{2} + 3123 \, n - 5715\right)} λ^{20} - \frac{7}{12288} \, {\left(5 \, n^{7} - 69 \, n^{6} + 297 \, n^{5} + 55 \, n^{4} - 3425 \, n^{3} + 5729 \, n^{2} + 3123 \, n - 5715\right)} λ^{19} - \frac{1}{36864} \, {\left(10 \, n^{8} + 349 \, n^{7} - 5929 \, n^{6} + 27201 \, n^{5} + 1255 \, n^{4} - 303769 \, n^{3} + 518789 \, n^{2} + 276219 \, n - 514125\right)} λ^{18} - \frac{1}{368640} \, {\left(1205 \, n^{8} + 1810 \, n^{7} - 153690 \, n^{6} + 846314 \, n^{5} - 258896 \, n^{4} - 8627850 \, n^{3} + 15586650 \, n^{2} + 7779726 \, n - 15175269\right)} λ^{17} - \frac{1}{368640} \, {\left(375 \, n^{9} + 3350 \, n^{8} - 26090 \, n^{7} - 231782 \, n^{6} + 1968180 \, n^{5} - 1702994 \, n^{4} - 17615110 \, n^{3} + 35375102 \, n^{2} + 15672645 \, n - 33443676\right)} λ^{16} - \frac{1}{1474560} \, {\left(625 \, n^{10} + 5660 \, n^{9} - 9825 \, n^{8} - 343840 \, n^{7} - 351878 \, n^{6} + 13590712 \, n^{5} - 22421394 \, n^{4} - 109304960 \, n^{3} + 259539973 \, n^{2} + 96052428 \, n - 236757501\right)} λ^{15} - \frac{1}{1474560} \, {\left(3515 \, n^{10} + 175 \, n^{9} - 114750 \, n^{8} - 367012 \, n^{7} + 1513458 \, n^{6} + 17057042 \, n^{5} - 47650672 \, n^{4} - 137507828 \, n^{3} + 406326107 \, n^{2} + 120694743 \, n - 360200538\right)} λ^{14} - \frac{1}{2949120} \, {\left(14575 \, n^{10} - 56570 \, n^{9} - 475829 \, n^{8} + 411688 \, n^{7} + 7240478 \, n^{6} + 29701860 \, n^{5} - 153062642 \, n^{4} - 290306648 \, n^{3} + 1155081907 \, n^{2} + 289740870 \, n - 979307289\right)} λ^{13} - \frac{1}{1474560} \, {\left(5985 \, n^{10} - 54590 \, n^{9} - 172057 \, n^{8} + 962968 \, n^{7} + 2770138 \, n^{6} + 7736172 \, n^{5} - 77009666 \, n^{4} - 104061608 \, n^{3} + 640062197 \, n^{2} + 1966818 \, n - 659106837\right)} λ^{12} + \frac{1}{23592960} \, {\left(20231 \, n^{10} + 240460 \, n^{9} + 314841 \, n^{8} - 6273648 \, n^{7} - 27169962 \, n^{6} - 164814712 \, n^{5} + 969837986 \, n^{4} + 1973776656 \, n^{3} - 7880568781 \, n^{2} - 2226225780 \, n + 6490675701\right)} λ^{11} + \frac{1}{23592960} \, {\left(31355 \, n^{10} - 174340 \, n^{9} - 356923 \, n^{8} - 3103536 \, n^{7} - 12732130 \, n^{6} - 43332440 \, n^{5} + 682291770 \, n^{4} + 932998224 \, n^{3} - 5532441417 \, n^{2} - 1524970692 \, n + 4165642161\right)} λ^{10} - \frac{1}{5898240} \, {\left(605 \, n^{10} + 51480 \, n^{9} + 91823 \, n^{8} - 889824 \, n^{7} - 4042782 \, n^{6} + 11072272 \, n^{5} - 25167202 \, n^{4} - 106995424 \, n^{3} + 464712849 \, n^{2} + 226621080 \, n - 286074909\right)} λ^{9} - \frac{1}{2949120} \, {\left(2385 \, n^{10} - 7530 \, n^{9} - 92043 \, n^{8} - 153480 \, n^{7} + 223442 \, n^{6} + 4336276 \, n^{5} - 6362334 \, n^{4} - 27617512 \, n^{3} + 94154461 \, n^{2} + 51852102 \, n - 52143255\right)} λ^{8} + \frac{1}{2949120} \, {\left(1155 \, n^{10} + 5580 \, n^{9} - 13851 \, n^{8} - 100448 \, n^{7} + 41406 \, n^{6} - 223784 \, n^{5} - 1322550 \, n^{4} + 336448 \, n^{3} - 8307441 \, n^{2} - 6284676 \, n + 385281\right)} λ^{7} - \frac{1}{1474560} \, {\left(81 \, n^{10} + 930 \, n^{9} + 2085 \, n^{8} + 3728 \, n^{7} + 13802 \, n^{6} + 96324 \, n^{5} + 90618 \, n^{4} - 369344 \, n^{3} + 682021 \, n^{2} + 526410 \, n - 284799\right)} λ^{6}

''

We may also write the same polynomials with a different coefficient order.

In [6]:
def clean_S(h):
    S = dict(compute_S(h))
    S2 = {}
    for j in S.keys():
        S2[j] = [p.polynomial(ring=PolynomialRing(QQ['λ'], 'n')) for p in S[j]]
    return S2

display_table(6, clean_S, "S", merge_even=sum)

***Even case***

S_1 = 2 λ

''

S_2 = \left(\frac{1}{4} λ^{3} - \frac{1}{8} λ^{2}\right) n^{2} - \frac{1}{4} λ^{2} n - λ^{3} - λ^{2}

''

...

S^2_5 = \left(-\frac{1}{960} λ^{12} - \frac{19}{3840} λ^{11} - \frac{171}{20480} λ^{10} - \frac{235}{98304} λ^{9} + \frac{63}{40960} λ^{8} + \frac{153}{81920} λ^{7} - \frac{11}{7680} λ^{6} + \frac{25}{98304} λ^{5}\right) n^{8} + \left(-\frac{1}{640} λ^{13} - \frac{73}{15360} λ^{12} + \frac{7}{3072} λ^{11} + \frac{1}{32} λ^{10} + \frac{39}{2048} λ^{9} + \frac{47}{5120} λ^{8} - \frac{27}{2560} λ^{7} - \frac{139}{30720} λ^{6} + \frac{37}{15360} λ^{5}\right) n^{7} + \left(-\frac{1}{3840} λ^{15} - \frac{1}{384} λ^{14} - \frac{3}{640} λ^{13} + \frac{377}{15360} λ^{12} + \frac{1991}{15360} λ^{11} + \frac{3259}{15360} λ^{10} + \frac{19493}{122880} λ^{9} + \frac{613}{10240} λ^{8} - \frac{893}{12288} λ^{7} + \frac{779}{61440} λ^{6} + \frac{679}{122880} λ^{5}\right) n^{6} + \left(\frac{7}{3840} λ^{15} + \frac{7}{384} λ^{14} + \frac{111}{1280} λ^{13} + \frac{949}{5120} λ^{12} + \frac{639}{5120} λ^{11} - \frac{827}{2560} λ^{10} + \frac{91}{6144} λ^{9} - \frac{3547}{15360} λ^{8} - \frac{7}{256} λ^{7} + \frac{2923}{30720} λ^{6} + \frac{77}{5120} λ^{5}\right) n^{5} + \left(-\frac{1}{640} λ^{15} - \frac{1}{64} λ^{14} - \frac{35}{256} λ^{13} - \frac{9991}{15360} λ^{12} - \frac{9713}{5120} λ^{11} - \frac{79723}{30720} λ^{10} - \frac{225731}{81920} λ^{9} - \frac{159697}{61440} λ^{8} - \frac{5401}{40960} λ^{7} + \frac{165}{4096} λ^{6} + \frac{12703}{245760} λ^{5}\right) n^{4} + \left(-\frac{7}{384} λ^{15} - \frac{35}{192} λ^{14} - \frac{29}{40} λ^{13} - \frac{21347}{15360} λ^{12} - \frac{3815}{3072} λ^{11} + \frac{2631}{1280} λ^{10} - \frac{83321}{30720} λ^{9} - \frac{3243}{5120} λ^{8} + \frac{2383}{7680} λ^{7} + \frac{681}{10240} λ^{6} + \frac{215}{3072} λ^{5}\right) n^{3} + \left(\frac{151}{3840} λ^{15} + \frac{151}{384} λ^{14} + \frac{287}{160} λ^{13} + \frac{26721}{5120} λ^{12} + \frac{184289}{15360} λ^{11} + \frac{248683}{15360} λ^{10} + \frac{1703473}{122880} λ^{9} + \frac{92987}{6144} λ^{8} + \frac{310291}{61440} λ^{7} + \frac{38083}{20480} λ^{6} + \frac{19567}{122880} λ^{5}\right) n^{2} + \left(\frac{21}{1280} λ^{15} + \frac{21}{128} λ^{14} + \frac{819}{1280} λ^{13} + \frac{6191}{5120} λ^{12} + \frac{24803}{15360} λ^{11} - \frac{28131}{2560} λ^{10} + \frac{254569}{30720} λ^{9} + \frac{161999}{15360} λ^{8} + \frac{12293}{1920} λ^{7} + \frac{62501}{30720} λ^{6} + \frac{2881}{15360} λ^{5}\right) n - \frac{3}{80} λ^{15} - \frac{3}{8} λ^{14} - \frac{423}{256} λ^{13} - \frac{23511}{5120} λ^{12} - \frac{9959}{1024} λ^{11} - \frac{471531}{20480} λ^{10} - \frac{152545}{32768} λ^{9} - \frac{37073}{40960} λ^{8} + \frac{313537}{81920} λ^{7} + \frac{26227}{20480} λ^{6} + \frac{42319}{163840} λ^{5}

''

S^1_6 = \left(\frac{9}{5120} λ^{17} + \frac{27}{2560} λ^{16} + \frac{7151}{294912} λ^{15} + \frac{7237}{294912} λ^{14} + \frac{4001}{589824} λ^{13} - \frac{1831}{491520} λ^{12} + \frac{29827}{23592960} λ^{11} + \frac{13787}{4718592} λ^{10} - \frac{995}{1179648} λ^{9} - \frac{21}{32768} λ^{8} + \frac{371}{983040} λ^{7} - \frac{9}{163840} λ^{6}\right) n^{10} + \left(\frac{3}{512} λ^{18} + \frac{3}{128} λ^{17} + \frac{163}{24576} λ^{16} - \frac{3779}{36864} λ^{15} - \frac{55859}{294912} λ^{14} - \frac{9287}{98304} λ^{13} + \frac{6541}{147456} λ^{12} + \frac{10523}{294912} λ^{11} - \frac{1957}{98304} λ^{10} - \frac{2725}{294912} λ^{9} + \frac{625}{98304} λ^{8} + \frac{13}{24576} λ^{7} - \frac{23}{49152} λ^{6}\right) n^{9} + \left(\frac{1}{512} λ^{20} + \frac{27}{1024} λ^{19} + \frac{1595}{18432} λ^{18} + \frac{4291}{73728} λ^{17} - \frac{6721}{36864} λ^{16} - \frac{101851}{294912} λ^{15} - \frac{23005}{147456} λ^{14} - \frac{7771}{327680} λ^{13} - \frac{298607}{1474560} λ^{12} - \frac{6025391}{23592960} λ^{11} - \frac{489569}{7864320} λ^{10} + \frac{248783}{5898240} λ^{9} + \frac{2381}{245760} λ^{8} - \frac{2341}{327680} λ^{7} + \frac{5}{98304} λ^{6}\right) n^{8} + \left(\frac{1}{3072} λ^{23} + \frac{1}{192} λ^{22} + \frac{1459}{36864} λ^{21} + \frac{1999}{12288} λ^{20} + \frac{1267}{4096} λ^{19} + \frac{317}{12288} λ^{18} - \frac{32899}{36864} λ^{17} - \frac{45805}{36864} λ^{16} + \frac{2131}{18432} λ^{15} + \frac{629717}{368640} λ^{14} + \frac{195781}{122880} λ^{13} + \frac{51071}{92160} λ^{12} + \frac{2707}{40960} λ^{11} + \frac{28721}{368640} λ^{10} + \frac{11509}{368640} λ^{9} - \frac{3847}{122880} λ^{8} - \frac{1051}{184320} λ^{7} - \frac{131}{46080} λ^{6}\right) n^{7} + \left(\frac{1}{46080} λ^{27} + \frac{7}{15360} λ^{26} + \frac{7}{1536} λ^{25} + \frac{133}{4608} λ^{24} + \frac{1}{8} λ^{23} + \frac{5581}{15360} λ^{22} + \frac{108977}{184320} λ^{21} - \frac{655}{12288} λ^{20} - \frac{33925}{12288} λ^{19} - \frac{243239}{36864} λ^{18} - \frac{1371517}{184320} λ^{17} - \frac{632579}{184320} λ^{16} + \frac{22889}{49152} λ^{15} + \frac{32219}{737280} λ^{14} - \frac{1539563}{1474560} λ^{13} + \frac{492163}{737280} λ^{12} + \frac{28008367}{11796480} λ^{11} + \frac{19804739}{11796480} λ^{10} + \frac{1147853}{2949120} λ^{9} + \frac{48397}{737280} λ^{8} + \frac{16033}{294912} λ^{7} - \frac{937}{147456} λ^{6}\right) n^{6} + \left(-\frac{1}{2880} λ^{27} - \frac{7}{960} λ^{26} - \frac{7}{96} λ^{25} - \frac{133}{288} λ^{24} - \frac{6299}{3072} λ^{23} - \frac{25429}{3840} λ^{22} - \frac{956039}{61440} λ^{21} - \frac{318613}{12288} λ^{20} - \frac{117057}{4096} λ^{19} - \frac{602461}{36864} λ^{18} + \frac{595429}{184320} λ^{17} + \frac{110777}{9216} λ^{16} + \frac{86629}{23040} λ^{15} - \frac{6420833}{737280} λ^{14} - \frac{10098047}{737280} λ^{13} - \frac{938123}{73728} λ^{12} - \frac{7285399}{737280} λ^{11} - \frac{4788901}{737280} λ^{10} - \frac{2240179}{737280} λ^{9} - \frac{582559}{737280} λ^{8} - \frac{1361}{5760} λ^{7} - \frac{23453}{368640} λ^{6}\right) n^{5} + \left(\frac{17}{9216} λ^{27} + \frac{119}{3072} λ^{26} + \frac{595}{1536} λ^{25} + \frac{11305}{4608} λ^{24} + \frac{351}{32} λ^{23} + \frac{111809}{3072} λ^{22} + \frac{3383425}{36864} λ^{21} + \frac{2192197}{12288} λ^{20} + \frac{1112453}{4096} λ^{19} + \frac{12265753}{36864} λ^{18} + \frac{31493437}{92160} λ^{17} + \frac{18946309}{61440} λ^{16} + \frac{63361507}{245760} λ^{15} + \frac{2184767}{10240} λ^{14} + \frac{258159457}{1474560} λ^{13} + \frac{30755503}{245760} λ^{12} + \frac{807177841}{11796480} λ^{11} + \frac{363593789}{11796480} λ^{10} + \frac{43703767}{2949120} λ^{9} + \frac{910777}{184320} λ^{8} + \frac{278651}{1474560} λ^{7} + \frac{98107}{737280} λ^{6}\right) n^{4} + \left(-\frac{1}{288} λ^{27} - \frac{7}{96} λ^{26} - \frac{35}{48} λ^{25} - \frac{665}{144} λ^{24} - \frac{63629}{3072} λ^{23} - \frac{26713}{384} λ^{22} - \frac{2219645}{12288} λ^{21} - \frac{1519681}{4096} λ^{20} - \frac{7585669}{12288} λ^{19} - \frac{31621579}{36864} λ^{18} - \frac{188630429}{

S^2_6 = \left(-\frac{125}{294912} λ^{15} - \frac{703}{294912} λ^{14} - \frac{2915}{589824} λ^{13} - \frac{133}{32768} λ^{12} + \frac{20231}{23592960} λ^{11} + \frac{6271}{4718592} λ^{10} - \frac{121}{1179648} λ^{9} - \frac{53}{65536} λ^{8} + \frac{77}{196608} λ^{7} - \frac{9}{163840} λ^{6}\right) n^{10} + \left(-\frac{25}{24576} λ^{16} - \frac{283}{73728} λ^{15} - \frac{35}{294912} λ^{14} + \frac{5657}{294912} λ^{13} + \frac{5459}{147456} λ^{12} + \frac{12023}{1179648} λ^{11} - \frac{8717}{1179648} λ^{10} - \frac{143}{16384} λ^{9} + \frac{251}{98304} λ^{8} + \frac{31}{16384} λ^{7} - \frac{31}{49152} λ^{6}\right) n^{9} + \left(-\frac{5}{18432} λ^{18} - \frac{241}{73728} λ^{17} - \frac{335}{36864} λ^{16} + \frac{655}{98304} λ^{15} + \frac{1275}{16384} λ^{14} + \frac{475829}{2949120} λ^{13} + \frac{172057}{1474560} λ^{12} + \frac{104947}{7864320} λ^{11} - \frac{356923}{23592960} λ^{10} - \frac{91823}{5898240} λ^{9} + \frac{10227}{327680} λ^{8} - \frac{1539}{327680} λ^{7} - \frac{139}{98304} λ^{6}\right) n^{8} + \left(-\frac{1}{36864} λ^{21} - \frac{5}{12288} λ^{20} - \frac{35}{12288} λ^{19} - \frac{349}{36864} λ^{18} - \frac{181}{36864} λ^{17} + \frac{2609}{36864} λ^{16} + \frac{2149}{9216} λ^{15} + \frac{91753}{368640} λ^{14} - \frac{51461}{368640} λ^{13} - \frac{120371}{184320} λ^{12} - \frac{43567}{163840} λ^{11} - \frac{64657}{491520} λ^{10} + \frac{9269}{61440} λ^{9} + \frac{1279}{24576} λ^{8} - \frac{3139}{92160} λ^{7} - \frac{233}{92160} λ^{6}\right) n^{7} + \left(\frac{23}{61440} λ^{21} + \frac{23}{4096} λ^{20} + \frac{161}{4096} λ^{19} + \frac{5929}{36864} λ^{18} + \frac{5123}{12288} λ^{17} + \frac{115891}{184320} λ^{16} + \frac{175939}{737280} λ^{15} - \frac{84081}{81920} λ^{14} - \frac{3620239}{1474560} λ^{13} - \frac{1385069}{737280} λ^{12} - \frac{4528327}{3932160} λ^{11} - \frac{1273213}{2359296} λ^{10} + \frac{224599}{327680} λ^{9} - \frac{111721}{1474560} λ^{8} + \frac{6901}{491520} λ^{7} - \frac{6901}{737280} λ^{6}\right) n^{6} + \left(-\frac{33}{20480} λ^{21} - \frac{99}{4096} λ^{20} - \frac{693}{4096} λ^{19} - \frac{9067}{12288} λ^{18} - \frac{423157}{184320} λ^{17} - \frac{32803}{6144} λ^{16} - \frac{1698839}{184320} λ^{15} - \frac{8528521}{737280} λ^{14} - \frac{495031}{49152} λ^{13} - \frac{644681}{122880} λ^{12} - \frac{20601839}{2949120} λ^{11} - \frac{1083311}{589824} λ^{10} - \frac{692017}{368640} λ^{9} - \frac{1084069}{737280} λ^{8} - \frac{27973}{368640} λ^{7} - \frac{8027}{122880} λ^{6}\right) n^{5} + \left(-\frac{11}{36864} λ^{21} - \frac{55}{12288} λ^{20} - \frac{385}{12288} λ^{19} - \frac{1255}{36864} λ^{18} + \frac{16181}{23040} λ^{17} + \frac{851497}{184320} λ^{16} + \frac{1245633}{81920} λ^{15} + \frac{2978167}{92160} λ^{14} + \frac{76531321}{1474560} λ^{13} + \frac{38504833}{737280} λ^{12} + \frac{484918993}{11796480} λ^{11} + \frac{22743059}{786432} λ^{10} + \frac{12583601}{2949120} λ^{9} + \frac{353463}{163840} λ^{8} - \frac{14695}{32768} λ^{7} - \frac{15103}{245760} λ^{6}\right) n^{4} + \left(\frac{685}{36864} λ^{21} + \frac{3425}{12288} λ^{20} + \frac{23975}{12288} λ^{19} + \frac{303769}{36864} λ^{18} + \frac{95865}{4096} λ^{17} + \frac{1761511}{36864} λ^{16} + \frac{170789}{2304} λ^{15} + \frac{34376957}{368640} λ^{14} + \frac{36288331}{368640} λ^{13} + \frac{13007701}{184320} λ^{12} + \frac{41120347}{491520} λ^{11} + \frac{19437463}{491520} λ^{10} + \frac{3343607}{184320} λ^{9} + \frac{3452189}{368640} λ^{8} + \frac{5257}{46080} λ^{7} + \frac{5771}{23040} λ^{6}\right) n^{3} + \left(-\frac{5729}{184320} λ^{21} - \frac{5729}{12288} λ^{20} - \frac{40103}{12288} λ^{19} - \frac{518789}{36864} λ^{18} - \frac{173185}{4096} λ^{17} - \frac{17687551}{184320} λ^{16} - \frac{259539973}{1474560} λ^{15} - \frac{406326107}{1474560} λ^{14} - \frac{1155081907}{2949120} λ^{13} - \frac{640062197}{1474560} λ^{12} - \frac{7880568781}{23592960} λ^{11} - \frac{614715713}{2621440} λ^{10} - \frac{51634761}{655360} λ^{9} - \frac{94154461}{2949120} λ^{8} - \frac{923049}{327680} λ^{7} - \frac{682021}{1474560} λ^{6}\

''

One may also easily compute $P_j(n) = S_j(n, 1) / 2^j$.

In [7]:
def compute_P(h):
    S = compute_S(h)
    P = {}
    for j in S.keys():
        P[j] = [(p / 2^j).subs(λ == 1).simplify_rational() for p in S[j]]
    return P

display_table(6, compute_P, "P", merge_even=sum)

***Even case***

P_1 = 1

''

P_2 = \frac{1}{32} \, n^{2} - \frac{1}{16} \, n - \frac{1}{2}

''

...

P^2_5 = -\frac{297}{655360} \, n^{8} + \frac{219}{163840} \, n^{7} + \frac{16061}{983040} \, n^{6} - \frac{121}{98304} \, n^{5} - \frac{54739}{163840} \, n^{4} - \frac{22531}{163840} \, n^{3} + \frac{734209}{327680} \, n^{2} + \frac{102983}{163840} \, n - \frac{811019}{655360}

''

P^1_6 = \frac{88047}{83886080} \, n^{10} - \frac{4803}{1048576} \, n^{9} - \frac{1323467}{83886080} \, n^{8} + \frac{463483}{11796480} \, n^{7} - \frac{85424029}{377487360} \, n^{6} - \frac{16250831}{7864320} \, n^{5} + \frac{2908510937}{75497472} \, n^{4} - \frac{1336086973}{11796480} \, n^{3} + \frac{699439151}{16777216} \, n^{2} + \frac{1813405027}{15728640} \, n - \frac{20167911149}{251658240}

P^2_6 = -\frac{2673}{16777216} \, n^{10} + \frac{3219}{4194304} \, n^{9} + \frac{468757}{83886080} \, n^{8} - \frac{72047}{9437184} \, n^{7} - \frac{29176489}{377487360} \, n^{6} - \frac{28007329}{31457280} \, n^{5} + \frac{1373345429}{377487360} \, n^{4} + \frac{83923433}{9437184} \, n^{3} - \frac{24960142337}{754974720} \, n^{2} - \frac{528110387}{62914560} \, n + \frac{7281010883}{251658240}

''

# Corollary 5.8: The polynomials F

Recall that 
$$F_j = k(k+1)\left[(1+\lambda)\frac{\partial}{\partial \lambda}S^1_j - j(k+1)S^1_j\right] + (n-k)(n-k+1) \left[(1+\lambda)\frac{\partial}{\partial \lambda}S^2_j - j(k+1)S^2_j\right],$$
which is a straightforward calculation in Sage.

In [8]:
def compute_F(h):
    S = compute_S(h)
    F = {}
    for j in S.keys():
        F[j] = clean(h, k*(k+1) * ((1+λ) * S[j][0].derivative(λ) - j*(k+1)*S[j][0]) +
                        (n-k)*(n-k+1)*((1+λ)*S[j][1].derivative(λ) - j*(k+1)*S[j][1]))
        assert QQ['λ']['n'](F[j]).degree() <= 2*j+1
        assert QQ['n']['λ'](F[j]).degree() <= 2*j^2
    return F

display_table(2, compute_F, "F")

***Even case***

F_1 = \frac{1}{2} \, n^{2} - \frac{1}{4} \, {\left(n^{3} + 2 \, n^{2}\right)} λ + n

''

F_2 = -\frac{1}{16} \, {\left(n^{5} + n^{4} - 6 \, n^{3} - 4 \, n^{2} + 8 \, n\right)} λ^{3} + \frac{1}{32} \, {\left(n^{5} + 10 \, n^{4} + 24 \, n^{3} - 8 \, n^{2} - 48 \, n\right)} λ^{2} - \frac{1}{16} \, {\left(n^{4} + 4 \, n^{3} + 12 \, n^{2} + 16 \, n\right)} λ

''

***Odd case***

F_1 = -\frac{1}{8} \, {\left(n^{3} + 3 \, n^{2} - n - 3\right)} λ^{2} + \frac{1}{2} \, n^{2} - \frac{1}{4} \, {\left(n^{3} + n^{2} - 5 \, n - 5\right)} λ + n + \frac{1}{2}

''

F_2 = -\frac{1}{32} \, {\left(n^{5} + 2 \, n^{4} - 6 \, n^{3} - 8 \, n^{2} + 5 \, n + 6\right)} λ^{5} - \frac{1}{32} \, {\left(2 \, n^{5} + n^{4} - 28 \, n^{3} - 34 \, n^{2} + 26 \, n + 33\right)} λ^{4} - \frac{1}{32} \, {\left(n^{5} - 8 \, n^{4} - 50 \, n^{3} - 32 \, n^{2} + 81 \, n + 72\right)} λ^{3} + \frac{1}{32} \, {\left(n^{5} + 8 \, n^{4} + 14 \, n^{3} - 32 \, n^{2} - 79 \, n - 40\right)} λ^{2} - \frac{1}{16} \, {\left(n^{4} + 4 \, n^{3} + 10 \, n^{2} + 12 \, n + 5\right)} λ

''

# Lemma 5.11: Computing a good $\lambda$ (and $B_1$, $\ldots$, $B_q$)

Recall that we want to count antichains of size $m$, and that the goal is to find $\lambda$ such that $|\mathbb{E}|\mathbf{A}| - m|$ is small.

For $N = \binom{n}{\lceil n/2 \rceil}$ and $\beta = m/N$ such that $1 - 4^{-1/t} + \epsilon \leq \beta \leq 1 - \epsilon$. By Corollary 5.8, it holds for q = $\lceil t/2 \rceil - 1$ that
$$\mathbb{E}|\mathbf{A}| = \frac{\lambda}{1+\lambda} N \left[1 + \frac{1}{(n-k+1)(k+1)}\sum_{j=1}^q F_j(n, \lambda)(1+\lambda)^{-j(k+1)}\right] + o(N^{1/2}).$$

If we assume our desired $\lambda$ is of the form
$$\lambda = \frac{\beta + X}{1-\beta}$$ and let $\gamma = (1-\beta)^{k+1}$, we may expand $(1-\lambda)^{-j(k+1)} = \gamma^j / (1+X)^{j(k+1)}$ as a function of $\gamma$ and $X$ in the expression of Corollary 5.8 to obtain
$$\frac{1}{N}\mathbb{E}|\mathbf{A}| = \left[(\beta+X)\sum_{i=0}^{q}(-X)^i\right] \cdot \left[1 + \frac{1}{(n-k+1)(k+1)}\sum_{j=1}^q F_j\left(n, \frac{\beta + X}{1-\beta}\right) \gamma^j \sum_{i=0}^q \binom{-j(k+1)}{i}X^i\right] + o(N^{1/2}).$$
The function `compute_E` below returns such an expression of $\gamma$, $\beta$ and $X$.

In [9]:
var("X β γ")

def compute_q(t):
    return (t+1)//2 - 1

def compute_E(h, t):
    F = compute_F(h)    
    q = compute_q(t)
    assert q in F

    # E is the expectation of |A|
    E = 1
    for j in range(1, q+1):
        # Expansion for (1+λ)^{-j(k+1)} by Newton's binomial theorem
        pow_lambda = γ^j * sum([binomial(-j*(k+1), i) * X^i for i in range(q+1)])
        # Sum the term corresponding to F[j]
        E += pow_lambda * F[j].subs(λ == (β + X)/(1-β))/((n-k+1) * (k+1))

    # Now we mulitply by λ/(1+λ) expanded in terms of β and X
    E *= (β + X) * sum([(-X)^i for i in range(q+1)])
    E = clean(h, E)

    return E

print("Taking t = 3 for the example")
display_table(2, lambda h: compute_E(h, 3), r"\frac{1}{N}\mathbb{E}|\mathbf{A}|")

Taking t = 3 for the example


***Even case***

\frac{1}{N}\mathbb{E}|\mathbf{A}| = -\frac{2 \, {\left({\left(X - 1\right)} n + 2 \, X - 2\right)} β^{2} - 4 \, X^{2} - 2 \, {\left(X^{2} - X\right)} n + 2 \, {\left(2 \, X^{2} + {\left(X^{2} - 2 \, X + 1\right)} n - 4 \, X + 2\right)} β - {\left({\left(X^{4} - X^{3}\right)} n^{3} + 2 \, {\left(X^{4} - 3 \, X^{3} + 2 \, X^{2}\right)} n^{2} + {\left({\left(X^{2} - X\right)} n^{3} + 2 \, {\left(2 \, X^{2} - 3 \, X + 1\right)} n^{2} + 4 \, {\left(X^{2} - 2 \, X + 1\right)} n\right)} β^{2} - 4 \, {\left(X^{3} - 2 \, X^{2} + X\right)} n + 2 \, {\left({\left(X^{3} - X^{2}\right)} n^{3} + 3 \, {\left(X^{3} - 2 \, X^{2} + X\right)} n^{2} + 2 \, {\left(X^{3} - 3 \, X^{2} + 3 \, X - 1\right)} n\right)} β\right)} γ + 4 \, X}{2 \, {\left({\left(n + 2\right)} β - n - 2\right)}}

***Odd case***

\frac{1}{N}\mathbb{E}|\mathbf{A}| = -\frac{4 \, {\left({\left(X - 1\right)} n + 3 \, X - 3\right)} β^{3} + 4 \, {\left(3 \, X^{2} + {\left(X^{2} - 3 \, X + 2\right)} n - 9 \, X + 6\right)} β^{2} + 12 \, X^{2} + 4 \, {\left(X^{2} - X\right)} n - 4 \, {\left(6 \, X^{2} + {\left(2 \, X^{2} - 3 \, X + 1\right)} n - 9 \, X + 3\right)} β - {\left(9 \, X^{5} + 15 \, X^{4} - {\left(X^{5} + X^{4} - 2 \, X^{3}\right)} n^{3} + {\left({\left(X^{2} - X\right)} n^{3} + {\left(5 \, X^{2} - 7 \, X + 2\right)} n^{2} - 9 \, X^{2} + {\left(3 \, X^{2} - 7 \, X + 4\right)} n + 15 \, X - 6\right)} β^{3} - 32 \, X^{3} - {\left(5 \, X^{5} - X^{4} - 12 \, X^{3} + 8 \, X^{2}\right)} n^{2} + {\left({\left(X^{3} - 3 \, X^{2} + 2 \, X\right)} n^{3} - 21 \, X^{3} + {\left(X^{3} - 17 \, X^{2} + 20 \, X - 4\right)} n^{2} + 41 \, X^{2} - {\left(13 \, X^{3} + 5 \, X^{2} - 34 \, X + 16\right)} n - 24 \, X + 4\right)} β^{2} - {\left(3 \, X^{5} - 17 \, X^{4} - 2 \, X^{3} + 24 \, X^{2} - 8 \, X\right)} n - {\left(3 \, X^{4} + {\left(X^{4} + 3 \, X^{3} - 4 \, X^{2}\right)} n^{3} - 41 \, X^{3} + 3 \, {\left(3 \, X^{4} + 3 \, X^{3} - 10 \, X^{2} + 4 \, X\right)} n^{2} + 50 \, X^{2} + {\left(19 \, X^{4} - 19 \, X^{3} - 32 \, X^{2} + 40 \, X - 8\right)} n - 4 \, X - 8\right)} β + 8 \, X\right)} γ - 12 \, X}{4 \, {\left({\left(n + 3\right)} β^{2} - 2 \, {\left(n + 3\right)} β + n + 3\right)}}

To obtain the desired rational functions $B_1, \ldots, B_q$, we now set
$$X := \lambda(1-\beta) - \beta = \sum_{j=1}^q B_j(n,\beta) (1-\beta)^{j(k+1) + 1}$$
and make the previously-obtained expression equal $\beta$ by solving for $B_1, \ldots, B_q$.

In [10]:
@memoized
def compute_B(h, t):
    E = compute_E(h, t)
    q = compute_q(t)
    
    # Now we expand X in terms of unknowns B_1, ..., B_q and replace it in E
    B = var("B", n=q+1)
    E = E.subs(X == sum([B[j] * γ^j * (1-β) for j in range(1, q+1)])).simplify_rational()

    substs = []
    coeffs = E.polynomial(ring=SR['γ']).coefficients(sparse=False)
    assert bool(coeffs[0] == β)

    B_ans = {}
    for j in range(1, q+1):
        ans = solve([coeffs[j].subs(substs) == 0], B[j])[0]
        B_ans[j] = B[j].subs(ans)
        substs.append(ans)

    return B_ans

print("Taking t = 8 for the example")
display_table(3, lambda h: compute_B(h, 8), "B")

Taking t = 8 for the example


***Even case***

B_1 = -\frac{{\left(n^{2} + 2 \, n\right)} β^{2} - 2 \, n β}{{\left(n + 2\right)} β^{3} - 3 \, {\left(n + 2\right)} β^{2} + 3 \, {\left(n + 2\right)} β - n - 2}

''

B_2 = \frac{{\left(n^{5} + 10 \, n^{4} + 28 \, n^{3} + 24 \, n^{2}\right)} β^{4} + {\left(n^{5} - 18 \, n^{4} - 72 \, n^{3} - 56 \, n^{2} + 16 \, n\right)} β^{3} - 32 \, n^{2} β - 2 \, {\left(n^{4} - 28 \, n^{3} - 36 \, n^{2} + 16 \, n\right)} β^{2}}{8 \, {\left({\left(n^{2} + 4 \, n + 4\right)} β^{5} - 5 \, {\left(n^{2} + 4 \, n + 4\right)} β^{4} + 10 \, {\left(n^{2} + 4 \, n + 4\right)} β^{3} - 10 \, {\left(n^{2} + 4 \, n + 4\right)} β^{2} - n^{2} + 5 \, {\left(n^{2} + 4 \, n + 4\right)} β - 4 \, n - 4\right)}}

...

''

B_2 = \frac{2 \, {\left(n^{4} + 6 \, n^{3} + 8 \, n^{2} - 6 \, n - 9\right)} β^{6} - {\left(n^{5} + 17 \, n^{4} + 62 \, n^{3} + 30 \, n^{2} - 95 \, n - 15\right)} β^{5} + 4 \, {\left(7 \, n^{4} + 38 \, n^{3} + 24 \, n^{2} - 70 \, n + 1\right)} β^{4} + {\left(n^{5} - 14 \, n^{4} - 164 \, n^{3} - 258 \, n^{2} + 227 \, n + 80\right)} β^{3} - 2 \, {\left(n^{4} - 26 \, n^{3} - 112 \, n^{2} - 38 \, n + 47\right)} β^{2} - 32 \, {\left(n^{2} + 2 \, n + 1\right)} β}{8 \, {\left({\left(n^{2} + 6 \, n + 9\right)} β^{7} - 7 \, {\left(n^{2} + 6 \, n + 9\right)} β^{6} + 21 \, {\left(n^{2} + 6 \, n + 9\right)} β^{5} - 35 \, {\left(n^{2} + 6 \, n + 9\right)} β^{4} + 35 \, {\left(n^{2} + 6 \, n + 9\right)} β^{3} - 21 \, {\left(n^{2} + 6 \, n + 9\right)} β^{2} - n^{2} + 7 \, {\left(n^{2} + 6 \, n + 9\right)} β - 6 \, n - 9\right)}}

''

B_3 = \frac{24 \, {\left(n^{7} + 3 \, n^{6} - 47 \, n^{5} - 309 \, n^{4} - 629 \, n^{3} - 207 \, n^{2} + 675 \, n + 513\right)} β^{10} + 3 \, {\left(3 \, n^{8} + 34 \, n^{7} + 626 \, n^{6} + 5018 \, n^{5} + 16792 \, n^{4} + 21606 \, n^{3} - 2706 \, n^{2} - 26658 \, n - 14715\right)} β^{9} - 2 \, {\left(9 \, n^{8} + 378 \, n^{7} + 5906 \, n^{6} + 34194 \, n^{5} + 80596 \, n^{4} + 51822 \, n^{3} - 62466 \, n^{2} - 78714 \, n - 31725\right)} β^{8} - 2 \, {\left(9 \, n^{8} - 468 \, n^{7} - 13280 \, n^{6} - 84084 \, n^{5} - 176230 \, n^{4} - 26652 \, n^{3} + 209352 \, n^{2} + 57444 \, n + 33909\right)} β^{7} + 12 \, {\left(3 \, n^{8} + 30 \, n^{7} - 2274 \, n^{6} - 20522 \, n^{5} - 49244 \, n^{4} - 2822 \, n^{3} + 69970 \, n^{2} + 274 \, n + 4585\right)} β^{6} + {\left(9 \, n^{8} - 1194 \, n^{7} + 10406 \, n^{6} + 207390 \, n^{5} + 699640 \, n^{4} + 283458 \, n^{3} - 1014150 \, n^{2} - 157878 \, n + 21471\right)} β^{5} - 2 \, {\left(9 \, n^{8} - 234 \, n^{7} - 890 \, n^{6} + 43470 \, n^{5} + 249032 \, n^{4} + 274146 \, n^{3} - 280326 \, n^{2} - 206790 \, n + 19887\right)} β^{4} + 12 \, {\left(3 \, n^{7} - 143 \, n^{6} + 927 \, n^{5} + 14677 \, n^{4} + 33625 \, n^{3} + 1379 \, n^{2} - 22267 \, n - 3625\right)} β^{3} + 384 \, {\left(3 \, n^{5} - 51 \, n^{4} - 286 \, n^{3} - 306 \, n^{2} + 27 \, n + 101\right)} β^{2} + 6144 \, {\left(n^{3} + 3 \, n^{2} + 3 \, n + 1\right)} β}{768 \, {\left({\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{11} - 11 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{10} + 55 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{9} - 165 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{8} + 330 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{7} - 462 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{6} + 462 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{5} - 330 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{4} + 165 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{3} - n^{3} - 55 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{2} - 9 \, n^{2} + 11 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β - 27 \, n - 27\right)}}

''

We package the obtained value of $X$ for convenience.

In [11]:
def compute_X(h, t):
    B = compute_B(h, t)
    q = compute_q(t)
    return sum([B[j] * γ^j * (1-β) for j in range(1, q+1)])

print("Taking t = 8 for the example")
display_table(3, lambda h: compute_X(h, 8), "X")

Taking t = 8 for the example


***Even case***

X = \frac{{\left(3 \, {\left(3 \, n^{8} + 46 \, n^{7} + 336 \, n^{6} + 1264 \, n^{5} + 2480 \, n^{4} + 2400 \, n^{3} + 896 \, n^{2}\right)} β^{7} + 3 \, {\left(3 \, n^{8} - 96 \, n^{7} - 1408 \, n^{6} - 5904 \, n^{5} - 10416 \, n^{4} - 7616 \, n^{3} - 1536 \, n^{2}\right)} β^{6} - {\left(9 \, n^{8} + 180 \, n^{7} - 5392 \, n^{6} - 32880 \, n^{5} - 59984 \, n^{4} - 30848 \, n^{3} + 6144 \, n^{2} - 1024 \, n\right)} β^{5} - {\left(9 \, n^{8} - 288 \, n^{7} + 1312 \, n^{6} + 28560 \, n^{5} + 66800 \, n^{4} + 26048 \, n^{3} - 21504 \, n^{2} + 4096 \, n\right)} β^{4} + 3072 \, n^{3} β + 6 \, {\left(3 \, n^{7} - 152 \, n^{6} + 1536 \, n^{5} + 7088 \, n^{4} + 3664 \, n^{3} - 3904 \, n^{2} + 512 \, n\right)} β^{3} + 192 \, {\left(3 \, n^{5} - 60 \, n^{4} - 76 \, n^{3} + 48 \, n^{2}\right)} β^{2}\right)} {\left(β - 1\right)} γ^{3}}{384 \, {\left({\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{8} - 8 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{7} + 28 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{6} - 56 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{5} + 70 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{4} - 56 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{3} + n^{3} + 28 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β^{2} + 6 \, n^{2} - 8 \, {\left(n^{3} + 6 \, n^{2} + 12 \, n + 8\right)} β + 12 \, n + 8\right)}} - \frac{{\left({\left(n^{5} + 10 \, n^{4} + 28 \, n^{3} + 24 \, n^{2}\right)} β^{4} + {\left(n^{5} - 18 \, n^{4} - 72 \, n^{3} - 56 \, n^{2} + 16 \, n\right)} β^{3} - 32 \, n^{2} β - 2 \, {\left(n^{4} - 28 \, n^{3} - 36 \, n^{2} + 16 \, n\right)} β^{2}\right)} {\left(β - 1\right)} γ^{2}}{8 \, {\left({\left(n^{2} + 4 \, n + 4\right)} β^{5} - 5 \, {\left(n^{2} + 4 \, n + 4\right)} β^{4} + 10 \, {\left(n^{2} + 4 \, n + 4\right)} β^{3} - 10 \, {\left(n^{2} + 4 \, n + 4\right)} β^{2} - n^{2} + 5 \, {\left(n^{2} + 4 \, n + 4\right)} β - 4 \, n - 4\right)}} + \frac{{\left({\left(n^{2} + 2 \, n\right)} β^{2} - 2 \, n β\right)} {\left(β - 1\right)} γ}{{\left(n + 2\right)} β^{3} - 3 \, {\left(n + 2\right)} β^{2} + 3 \, {\left(n + 2\right)} β - n - 2}

***Odd case***

X = -\frac{{\left(24 \, {\left(n^{7} + 3 \, n^{6} - 47 \, n^{5} - 309 \, n^{4} - 629 \, n^{3} - 207 \, n^{2} + 675 \, n + 513\right)} β^{10} + 3 \, {\left(3 \, n^{8} + 34 \, n^{7} + 626 \, n^{6} + 5018 \, n^{5} + 16792 \, n^{4} + 21606 \, n^{3} - 2706 \, n^{2} - 26658 \, n - 14715\right)} β^{9} - 2 \, {\left(9 \, n^{8} + 378 \, n^{7} + 5906 \, n^{6} + 34194 \, n^{5} + 80596 \, n^{4} + 51822 \, n^{3} - 62466 \, n^{2} - 78714 \, n - 31725\right)} β^{8} - 2 \, {\left(9 \, n^{8} - 468 \, n^{7} - 13280 \, n^{6} - 84084 \, n^{5} - 176230 \, n^{4} - 26652 \, n^{3} + 209352 \, n^{2} + 57444 \, n + 33909\right)} β^{7} + 12 \, {\left(3 \, n^{8} + 30 \, n^{7} - 2274 \, n^{6} - 20522 \, n^{5} - 49244 \, n^{4} - 2822 \, n^{3} + 69970 \, n^{2} + 274 \, n + 4585\right)} β^{6} + {\left(9 \, n^{8} - 1194 \, n^{7} + 10406 \, n^{6} + 207390 \, n^{5} + 699640 \, n^{4} + 283458 \, n^{3} - 1014150 \, n^{2} - 157878 \, n + 21471\right)} β^{5} - 2 \, {\left(9 \, n^{8} - 234 \, n^{7} - 890 \, n^{6} + 43470 \, n^{5} + 249032 \, n^{4} + 274146 \, n^{3} - 280326 \, n^{2} - 206790 \, n + 19887\right)} β^{4} + 12 \, {\left(3 \, n^{7} - 143 \, n^{6} + 927 \, n^{5} + 14677 \, n^{4} + 33625 \, n^{3} + 1379 \, n^{2} - 22267 \, n - 3625\right)} β^{3} + 384 \, {\left(3 \, n^{5} - 51 \, n^{4} - 286 \, n^{3} - 306 \, n^{2} + 27 \, n + 101\right)} β^{2} + 6144 \, {\left(n^{3} + 3 \, n^{2} + 3 \, n + 1\right)} β\right)} {\left(β - 1\right)} γ^{3}}{768 \, {\left({\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{11} - 11 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{10} + 55 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{9} - 165 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{8} + 330 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{7} - 462 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{6} + 462 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{5} - 330 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{4} + 165 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{3} - n^{3} - 55 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β^{2} - 9 \, n^{2} + 11 \, {\left(n^{3} + 9 \, n^{2} + 27 \, n + 27\right)} β - 27 \, n - 27\right)}} - \frac{{\left(2 \, {\left(n^{4} + 6 \, n^{3} + 8 \, n^{2} - 6 \, n - 9\right)} β^{6} - {\left(n^{5} + 17 \, n^{4} + 62 \, n^{3} + 30 \, n^{2} - 95 \, n - 15\right)} β^{5} + 4 \, {\left(7 \, n^{4} + 38 \, n^{3} + 24 \, n^{2} - 70 \, n + 1\right)} β^{4} + {\left(n^{5} - 14 \, n^{4} - 164 \, n^{3} - 258 \, n^{2} + 227 \, n + 80\right)} β^{3} - 2 \, {\left(n^{4} - 26 \, n^{3} - 112 \, n^{2} - 38 \, n + 47\right)} β^{2} - 32 \, {\left(n^{2} + 2 \, n + 1\right)} β\right)} {\left(β - 1\right)} γ^{2}}{8 \, {\left({\left(n^{2} + 6 \, n + 9\right)} β^{7} - 7 \, {\left(n^{2} + 6 \, n + 9\right)} β^{6} + 21 \, {\left(n^{2} + 6 \, n + 9\right)} β^{5} - 35 \, {\left(n^{2} + 6 \, n + 9\right)} β^{4} + 35 \, {\left(n^{2} + 6 \, n + 9\right)} β^{3} - 21 \, {\left(n^{2} + 6 \, n + 9\right)} β^{2} - n^{2} + 7 \, {\left(n^{2} + 6 \, n + 9\right)} β - 6 \, n - 9\right)}} + \frac{{\left({\left(n^{2} + 2 \, n - 3\right)} β^{3} - 2 \, {\left(n^{2} + 4 \, n - 1\right)} β^{2} + 4 \, {\left(n + 1\right)} β\right)} {\left(β - 1\right)} γ}{2 \, {\left({\left(n + 3\right)} β^{4} - 4 \, {\left(n + 3\right)} β^{3} + 6 \, {\left(n + 3\right)} β^{2} - 4 \, {\left(n + 3\right)} β + n + 3\right)}}

# Theorem 5.1: The rational functions $R_j$

We want to obtain the coefficients of
$$\log\left(\frac{i_m^C}{\binom{N}{m}}\right) = \sum_{j=1}^{t-1}\left[N \sum_{j=1}^{t-1}R_j(n, \beta) (1-\beta)^{jk}\right].$$
Recall that
$$\Xi_C = (1+o(1))\exp\left[\sum_{j=1}^{t-1}\binom{n}{k-1}S^1_j(n, \lambda) + \sum_{j=1}^{t-1}\binom{n}{k+1}S^2_j(n, \lambda)\right].$$
and that, for $\lambda_0 = \beta/(1-\beta)$,
$$i_m^C = (1+o(1)) \binom{N}{m} \left(\frac{1+\lambda}{1+\lambda_0}\right)^N \left(\frac{\lambda}{\lambda_0}\right)^N \Xi_C.$$
Taking logs, dividing by $N = \binom{n}{k}$ and Taylor expanding, we may write
$$\log\left(\frac{i_m^C}{\binom{N}{m}}\right) = \sum_{i=1}^{t-1} \frac{(-1)^{i+1}}{i} (1 - \beta^{1-i}) X^i + \sum_{j=1}^{t-1} \left(\frac{k}{n-k+1}S^1_j\left(n, \frac{\beta+X}{1-\beta}\right) + \frac{n-k}{k+1}S^2_j\left(n, \frac{\beta+X}{1-\beta}\right)\right) \gamma^j \sum_{i=0}^{t-1}\binom{-j(k+1)}{i}X^i + o(N^{1/2}).$$
Recall that we already solved for $X$ as a function of $n$, $\beta$ and $\gamma$. Substituting, it only remains to collect the coefficients of $\gamma$. Up to multiplying by $(1-\beta)^j$ (to correct for the fact that $\gamma = (1-\beta)^{k+1}$ while the $R_j$ are the coefficients of a power series in $(1-\beta)^{k}$), these are our $R_j$s.

In [12]:
@memoized
def compute_R(h):
    S = compute_S(h)
    t = 1 + max(S.keys())

    # The first part of the above equation, as per 5.22
    taylr = sum([(-1)^(i+1)/i * (1-β^(1-i)) * X^i for i in range(1, t)])
    
    # To consider the remaining terms, we expand (1+λ)^{-j(k+1)} 
    # by Newton's binomial theorem, as before
    for j in range(1, t):
        pow_lambda = γ^j * sum([binomial(-j*(k+1), i) * X^i for i in range(t)])
        second_sum = k / (n - k + 1) * S[j][0] + (n - k) / (k + 1) * S[j][1]
        taylr += second_sum.subs(λ == (β+X)/(1-β)) * pow_lambda
    taylr = clean(h, taylr).subs(X == compute_X(h, t))

    # It remains to compare coefficients and multiply by (1-β)^j
    # TODO: Why is computing coeffs so slow? Is expanding X^i the bottleneck?
    print("Collecting coefficients")
    coeffs = taylr.polynomial(ring=SR['γ']).list()
    assert bool(coeffs[0] == 0)
    
    R = {}
    for j in range(1, t):
        R[j] = (coeffs[j] * (1-β)^j).simplify_rational()
    return R

display_table(6, compute_R, "R")

***Even case***

R_1 = \frac{2 \, n β}{n + 2}

''

R_2 = -\frac{{\left(n^{4} + 8 \, n^{3} + 12 \, n^{2}\right)} β^{3} + 16 \, n^{2} β + {\left(n^{4} - 12 \, n^{3} - 20 \, n^{2} + 16 \, n\right)} β^{2}}{8 \, {\left(n^{2} - {\left(n^{2} + 4 \, n + 4\right)} β + 4 \, n + 4\right)}}

''

...

''

R_5 = \frac{64 \, {\left(15 \, n^{11} - 125 \, n^{10} - 3955 \, n^{9} - 29351 \, n^{8} - 87322 \, n^{7} + 83758 \, n^{6} + 1589546 \, n^{5} + 5298306 \, n^{4} + 7044219 \, n^{3} - 31617 \, n^{2} - 8542503 \, n - 5320971\right)} β^{20} - 5 \, {\left(n^{13} - 33 \, n^{12} + 454 \, n^{11} - 47678 \, n^{10} - 698749 \, n^{9} - 3407051 \, n^{8} - 2633180 \, n^{7} + 49673260 \, n^{6} + 285021367 \, n^{5} + 748360857 \, n^{4} + 878591286 \, n^{3} - 97103502 \, n^{2} - 1160281179 \, n - 697475853\right)} β^{19} + 10 \, {\left(15 \, n^{13} + 273 \, n^{12} + 4442 \, n^{11} - 117538 \, n^{10} - 1423635 \, n^{9} - 2420581 \, n^{8} + 33627548 \, n^{7} + 251754900 \, n^{6} + 899705657 \, n^{5} + 1898124023 \, n^{4} + 1932533226 \, n^{3} - 488350674 \, n^{2} - 2864447253 \, n - 1658990403\right)} β^{18} - 5 \, {\left(155 \, n^{13} + 5861 \, n^{12} + 112482 \, n^{11} + 205110 \, n^{10} + 1995729 \, n^{9} + 74874167 \, n^{8} + 630873292 \, n^{7} + 2635298084 \, n^{6} + 6847638477 \, n^{5} + 11688739267 \, n^{4} + 10109315634 \, n^{3} - 4583016250 \, n^{2} - 17589935769 \, n - 9816106239\right)} β^{17} + {\left(1185 \, n^{13} + 97595 \, n^{12} + 2709590 \, n^{11} + 27145850 \, n^{10} + 309651315 \, n^{9} + 2807185945 \, n^{8} + 14579139140 \, n^{7} + 44573263932 \, n^{6} + 88184221623 \, n^{5} + 120983054205 \, n^{4} + 86875793030 \, n^{3} - 66401841110 \, n^{2} - 189753335019 \, n - 102187087281\right)} β^{16} + 4 \, {\left(285 \, n^{13} - 28790 \, n^{12} - 1611030 \, n^{11} - 27371808 \, n^{10} - 350717421 \, n^{9} - 2665561102 \, n^{8} - 11245882804 \, n^{7} - 27180189776 \, n^{6} - 40622099245 \, n^{5} - 43456381586 \, n^{4} - 25483159062 \, n^{3} + 33746624176 \, n^{2} + 77112072413 \, n + 40174305750\right)} β^{15} - 5 \, {\left(1023 \, n^{13} + 18365 \, n^{12} - 1406326 \, n^{11} - 46027914 \, n^{10} - 727046643 \, n^{9} - 5439877521 \, n^{8} - 20683993476 \, n^{7} - 41121782044 \, n^{6} - 43624857879 \, n^{5} - 32616941045 \, n^{4} - 16461794854 \, n^{3} + 41848821190 \, n^{2} + 78960495659 \, n + 39914391465\right)} β^{14} + 20 \, {\left(152 \, n^{13} + 20057 \, n^{12} + 67332 \, n^{11} - 13449846 \, n^{10} - 313884260 \, n^{9} - 2538210241 \, n^{8} - 9365403000 \, n^{7} - 16008970068 \, n^{6} - 10423950928 \, n^{5} - 2522706969 \, n^{4} - 2384793612 \, n^{3} + 13050413930 \, n^{2} + 20473125532 \, n + 10047741921\right)} β^{13} + 10 \, {\left(561 \, n^{13} - 33781 \, n^{12} - 1406122 \, n^{11} + 10989946 \, n^{10} + 747822851 \, n^{9} + 7167408361 \, n^{8} + 27452739780 \, n^{7} + 43194534076 \, n^{6} + 12888364391 \, n^{5} - 15920740115 \, n^{4} + 2488230150 \, n^{3} - 26245443414 \, n^{2} - 35197105083 \, n - 16585361601\right)} β^{12} - {\left(7885 \, n^{13} + 163515 \, n^{12} - 16944530 \, n^{11} - 169861590 \, n^{10} + 5960025415 \, n^{9} + 77190794825 \, n^{8} + 327622643860 \, n^{7} + 519080338076 \, n^{6} + 40790346827 \, n^{5} - 408298019875 \, n^{4} + 1345399710 \, n^{3} - 194828466150 \, n^{2} - 256478387967 \, n - 112097376705\right)} β^{11} - 8 \, {\left(30 \, n^{13} - 59795 \, n^{12} + 723800 \, n^{11} + 42410962 \, n^{10} - 334923122 \, n^{9} - 7848195077 \, n^{8} - 39434388864 \, n^{7} - 68845436452 \, n^{6} - 3003491334 \, n^{5} + 73649888755 \, n^{4} + 8325346536 \, n^{3} + 9862436082 \, n^{2} + 19710176922 \, n + 7749682437\right)} β^{10} + 5 \, {\left(1149 \, n^{13} - 51893 \, n^{12} - 1322418 \, n^{11} + 57648458 \, n^{10} + 10334103 \, n^{9} - 7483238599 \, n^{8} - 48057080108 \, n^{7} - 99320062884 \, n^{6} - 17974458469 \, n^{5} + 119228897101 \, n^{4} + 35444180862 \, n^{3} - 2851871430 \, n^{2} + 14314931121 \, n + 5726123343\right)} β^{9} - 5 \, {\left(543 \, n^{13} + 16069 \, n^{12} - 1829462 \, n^{11} + 26631430 \, n^{10} + 197617997 \, n^{9} - 3051309977 \, n^{8} - 28130179140 \, n^{7} - 72552011068 \, n^{6} - 32545596279 \, n^{5} + 84920414595 \, n^{4} + 50229891450 \, n^{3} - 5793726122 \, n^{2} + 2077493547 \, n + 2206667313\right)} β^{8} - 10 \, {\left(101 \, n^{13} - 15539 \, n^{12} + 485830 \, n^{11

''

R_6 = \frac{3072 \, {\left(20 \, n^{13} + 293 \, n^{12} - 866 \, n^{11} - 30992 \, n^{10} - 189010 \, n^{9} - 525083 \, n^{8} - 507540 \, n^{7} + 3450824 \, n^{6} + 25620304 \, n^{5} + 78387135 \, n^{4} + 100992726 \, n^{3} - 5166072 \, n^{2} - 125915634 \, n - 76116105\right)} β^{27} - 3 \, {\left(n^{16} - 142 \, n^{15} - 4590 \, n^{14} + 162690 \, n^{13} + 1154262 \, n^{12} - 51981318 \, n^{11} - 684707350 \, n^{10} - 3166843670 \, n^{9} - 5239145688 \, n^{8} + 9232267734 \, n^{7} + 97395886806 \, n^{6} + 456752073798 \, n^{5} + 1254181048650 \, n^{4} + 1561891504446 \, n^{3} - 129494685906 \, n^{2} - 2024657183538 \, n - 1216159546185\right)} β^{26} + 30 \, {\left(9 \, n^{16} + 54 \, n^{15} - 3326 \, n^{14} + 27246 \, n^{13} - 1679954 \, n^{12} - 63933986 \, n^{11} - 596492630 \, n^{10} - 2051694250 \, n^{9} + 158643008 \, n^{8} + 23751208962 \, n^{7} + 108419490694 \, n^{6} + 374705460330 \, n^{5} + 926141669874 \, n^{4} + 1113333008490 \, n^{3} - 133514595378 \, n^{2} - 1509674076846 \, n - 900607032297\right)} β^{25} - 30 \, {\left(103 \, n^{16} + 2922 \, n^{15} + 32690 \, n^{14} + 348234 \, n^{13} - 13522562 \, n^{12} - 357283214 \, n^{11} - 2665819894 \, n^{10} - 4962204174 \, n^{9} + 31636443372 \, n^{8} + 215894626942 \, n^{7} + 703935660086 \, n^{6} + 1924896249950 \, n^{5} + 4297676389426 \, n^{4} + 4990427847206 \, n^{3} - 808470514674 \, n^{2} - 7125899587866 \, n - 4222098668547\right)} β^{24} + 30 \, {\left(479 \, n^{16} + 22634 \, n^{15} + 482818 \, n^{14} + 6152490 \, n^{13} - 12943474 \, n^{12} - 894212302 \, n^{11} - 4807971366 \, n^{10} + 17531010066 \, n^{9} + 268121544748 \, n^{8} + 1179912618494 \, n^{7} + 3091483423398 \, n^{6} + 6940588103550 \, n^{5} + 14056933189794 \, n^{4} + 15823541920998 \, n^{3} - 3300876946818 \, n^{2} - 23960685615930 \, n - 14110840779579\right)} β^{23} - 6 \, {\left(4943 \, n^{16} + 404762 \, n^{15} + 12735250 \, n^{14} + 213638490 \, n^{13} + 1390001118 \, n^{12} + 4449325218 \, n^{11} + 76192714794 \, n^{10} + 1047372297730 \, n^{9} + 6581592572444 \, n^{8} + 22589459866638 \, n^{7} + 49657535904726 \, n^{6} + 93285695979022 \, n^{5} + 172298057166610 \, n^{4} + 189396166164662 \, n^{3} - 49040323686930 \, n^{2} - 306328202097642 \, n - 179569612991835\right)} β^{22} + 2 \, {\left(2757 \, n^{16} + 2104230 \, n^{15} + 109745070 \, n^{14} + 2487285990 \, n^{13} + 27592214002 \, n^{12} + 244381502238 \, n^{11} + 2176651842262 \, n^{10} + 15184182300990 \, n^{9} + 68926873932796 \, n^{8} + 195917146130994 \, n^{7} + 366848495204586 \, n^{6} + 581097224745138 \, n^{5} + 986091114413310 \, n^{4} + 1070671014866058 \, n^{3} - 334530092144238 \, n^{2} - 1863371746220118 \, n - 1089285437926065\right)} β^{21} + 30 \, {\left(3291 \, n^{16} - 31118 \, n^{15} - 11553030 \, n^{14} - 403699886 \, n^{13} - 6297407146 \, n^{12} - 70541948902 \, n^{11} - 584755678670 \, n^{10} - 3320352008678 \, n^{9} - 12387742331156 \, n^{8} - 29831329883498 \, n^{7} - 47515360712818 \, n^{6} - 63415337704074 \, n^{5} - 100090782258342 \, n^{4} - 108932284314786 \, n^{3} + 40336285445190 \, n^{2} + 205709680838814 \, n + 120109233244809\right)} β^{20} - 10 \, {\left(17823 \, n^{16} + 1080426 \, n^{15} - 16151310 \, n^{14} - 1840505718 \, n^{13} - 42203939746 \, n^{12} - 575992592910 \, n^{11} - 4823052535990 \, n^{10} - 24953512960590 \, n^{9} - 81120251298436 \, n^{8} - 166706912802690 \, n^{7} - 221920507776522 \, n^{6} - 247180746860514 \, n^{5} - 371871159920238 \, n^{4} - 411314116866138 \, n^{3} + 180210136660686 \, n^{2} + 852852335547174 \, n + 497447840904693\right)} β^{19} + 40 \, {\left(576 \, n^{16} + 516579 \, n^{15} + 12696657 \, n^{14} - 340955481 \, n^{13} - 16045171691 \, n^{12} - 282337445433 \, n^{11} - 2513126655923 \, n^{10} - 12560253812277 \, n^{9} - 37012780882895 \, n^{8} - 64989410072583 \, n^{7} - 69302419503741 \, n^{6} - 62692149471195 \, n^{5} - 95393839518609 \, n^{4} - 107487833167203 \, n^{3} + 57167935212447 \, n^{2} + 249868398299433 \, n + 145214189931339\right)} β^{1

''

The denominators above are a bit ugly. Let us factor them. The expressions $R_i$ above equal the $N_i/D_i$ computed below.

In [23]:
def simplify_R(h):
    R = compute_R(h)
    N, D = {}, {}
    for j in R.keys():
        N[j] = R[j].numerator().simplify_full()
        D[j] = R[j].denominator().factor()
    return N, D

display_table(6, simplify_R, "N,D")

***Even case***

N_1 = 2 \, n β

''

N_2 = {\left(n^{4} + 8 \, n^{3} + 12 \, n^{2}\right)} β^{3} + 16 \, n^{2} β + {\left(n^{4} - 12 \, n^{3} - 20 \, n^{2} + 16 \, n\right)} β^{2}

''

...

''

D_5 = 491520 \, {\left(n + 3\right)}^{5} {\left(β - 1\right)}^{15}

''

D_6 = 11796480 \, {\left(n + 3\right)}^{6} {\left(β - 1\right)}^{21}

''